In [5]:
import os
from typing import Literal, TypedDict

import gspread
import pandas as pd
from dotenv import load_dotenv

from helpers import upload_to_github

load_dotenv()

CREDENTIALS_INFO = {
    "type": "service_account",
    "project_id": os.getenv("PROJECT_ID"),
    "private_key_id": os.getenv("PRIVATE_KEY_ID"),
    "private_key": os.getenv("PRIVATE_KEY"),
    "client_email": os.getenv("CLIENT_EMAIL"),
    "client_id": os.getenv("CLIENT_ID"),
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": os.getenv("CLIENT_x509_CERT_URL"),
    "universe_domain": "googleapis.com",
}

SHEET_URL = os.getenv("SHEET_URL", "")


class WorksheetRangePairing(TypedDict):
    worksheet_id: str
    range_name: str
    date: str  # in yyyy-mm-dd format
    clean_script_version: Literal["group_based", "ffa"]


WORKSHEET_RANGE_PAIRING: list[WorksheetRangePairing] = [
    {
        "worksheet_id": "1059373094",
        "range_name": "A10:D42",
        "date": "2024-10-10",
        "clean_script_version": "group_based",
    },
    {
        "worksheet_id": "2144589541",
        "range_name": "A10:D50",
        "date": "2024-10-17",
        "clean_script_version": "group_based",
    },
    {
        "worksheet_id": "443320611",
        "range_name": "A5:D53",
        "date": "2024-10-31",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "1717508104",
        "range_name": "A5:D45",
        "date": "2024-11-07",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "2019248424",
        "range_name": "A5:D63",
        "date": "2024-11-14",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "1979486117",
        "range_name": "A5:D39",
        "date": "2024-11-21",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "1529201747",
        "range_name": "A5:D57",
        "date": "2024-11-28",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "948123286",
        "range_name": "A5:D49",
        "date": "2024-12-05",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "1638262922",
        "range_name": "A5:D41",
        "date": "2024-12-12",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "1440311102",
        "range_name": "A5:D41",
        "date": "2025-01-02",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "421923177",
        "range_name": "A5:D33",
        "date": "2025-01-09",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "180922140",
        "range_name": "A5:D41",
        "date": "2025-01-16",
        "clean_script_version": "ffa",
    },
    {
        "worksheet_id": "2084155020",
        "range_name": "A5:D43",
        "date": "2025-01-23",
        "clean_script_version": "ffa",
    },
]


def read_public_google_sheet_to_dataframe(
    sheet_url: str, worksheet_id: str, range_name: str
) -> pd.DataFrame:
    """
    Reads data from a public Google Sheet into a pandas DataFrame.

    Args:
        sheet_url (str): The URL of the public Google Sheet.
        range_name (str): The range to read, e.g., 'Sheet1!A1:C10'.

    Returns:
        pd.DataFrame: Data from the specified range in the Google Sheet as a DataFrame.
    """

    client = gspread.service_account_from_dict(info=CREDENTIALS_INFO)
    sheet = client.open_by_url(sheet_url)

    # Get the worksheet data
    worksheet = sheet.get_worksheet_by_id(
        worksheet_id
    )  # Access the first sheet or specify by name

    data = worksheet.get(range_name)

    # Convert the data into a DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])  # Assuming the first row is header

    return df


def clean_data_group_based(df: pd.DataFrame) -> pd.DataFrame:
    _df = df.copy()

    # Step 1: Forward fill 'Match No.' to group teams by each match
    _df["Match No."] = _df["Match No."].ffill()

    # Step 2: Split Players column into separate player names
    _df[["Player_1", "Player_2"]] = _df["Players"].str.split(" & ", expand=True)

    # Step 3: Split the data into Team A and Team B based on even-odd index within each match
    df_a = _df.iloc[::2].copy().reset_index(drop=True)
    df_b = _df.iloc[1::2].copy().reset_index(drop=True)

    # Step 4: Rename columns in df_a and df_b for merging
    df_a = df_a.rename(
        columns={"Player_1": "player_a_1", "Player_2": "player_a_2", "Score": "score_a"}
    ).drop(columns=["Players", "Team"])
    df_b = df_b.rename(
        columns={"Player_1": "player_b_1", "Player_2": "player_b_2", "Score": "score_b"}
    ).drop(columns=["Players", "Team", "Match No."])

    # Step 5: Combine df_a and df_b side by side to get the final format
    df_final = pd.concat([df_a, df_b], axis=1)
    df_final = df_final[
        ["player_a_1", "player_a_2", "player_b_1", "player_b_2", "score_a", "score_b"]
    ]
    return df_final


def clean_data_ffa(df: pd.DataFrame) -> pd.DataFrame:
    # Step 1: Forward fill 'Match No.' to group each match's teams together
    df["Match No."] = df["Match No."].ffill()

    # Step 2: Split the data into Team A and Team B based on even-odd index within each match
    df_a = df.iloc[::2].reset_index(drop=True)  # Team A (even rows)
    df_b = df.iloc[1::2].reset_index(drop=True)  # Team B (odd rows)

    # Step 3: Rename columns for Team A and Team B
    df_a = df_a.rename(
        columns={"Player 1": "player_a_1", "Player 2": "player_a_2", "Score": "score_a"}
    )
    df_b = df_b.rename(
        columns={"Player 1": "player_b_1", "Player 2": "player_b_2", "Score": "score_b"}
    )

    # Step 4: Drop unnecessary columns in df_b and merge the dataframes side-by-side
    df_b = df_b.drop(columns=["Match No."])
    df_final = pd.concat([df_a, df_b], axis=1)
    df_final = df_final[
        ["player_a_1", "player_a_2", "player_b_1", "player_b_2", "score_a", "score_b"]
    ]

    return df_final


for item in WORKSHEET_RANGE_PAIRING:
    df = read_public_google_sheet_to_dataframe(
        SHEET_URL, worksheet_id=item["worksheet_id"], range_name=item["range_name"]
    )
    if item["clean_script_version"] == "group_based":
        clean_df = clean_data_group_based(df)
    else:
        clean_df = clean_data_ffa(df)

    file_path = f"matches/{item['date']}_match_results.csv"

    clean_df.to_csv(f"./{file_path}", index=False)

    upload_to_github(
        f"./{file_path}", commit_message=f"Upload matches data for {item['date']}"
    )

    display(
        pd.read_csv(
            f"https://raw.githubusercontent.com/BadgerMinton/badgerminton-data/refs/heads/main/{file_path}"
        )
    )

File exists! Uploading...
Updated ./matches/2024-10-10_match_results.csv in badgerminton/badgerminton-data!


,player_a_1,player_a_2,player_b_1,player_b_2,score_a,score_b
0,Isha,Kamil,Mujahid,Luqman,21,12
1,Mirza,Hasanah,Khairul,Suraya,8,21
2,Afiqah,Rafiq,Luqman,Nadia,21,13
3,Alif,Maisarah,Shazwan,Hazwan,16,21
4,Khairul,Suraya,Isha,Kamil,21,18
5,Mirza,Hasanah,Mujahid,Yassier,18,21
6,Shazwan,Hazwan,Luqman,Nadia,21,18
7,Alif,Maisarah,Afiqah,Rafiq,17,21
8,Isha,Kamil,Mirza,Hasanah,21,11
9,Alif,Maisarah,Yassier,Mujahid,21,18


File exists! Uploading...
Updated ./matches/2024-10-17_match_results.csv in badgerminton/badgerminton-data!


,player_a_1,player_a_2,player_b_1,player_b_2,score_a,score_b
0,Mujahid,Lisa,Mirza,Suhailah,21,13
1,Afiqah,Zulhakim,Khairul,Shazwan,13,21
2,Isha,Hazwan,Rushdi,Nadia,21,15
3,Luqman,Dina,Afiqah,Ammar,13,21
4,Khairul,Shazwan,Mujahid,Lisa,21,15
5,Afiqah,Zulhakim,Mirza,Suhailah,21,17
6,Yassier,Ammar,Rushdi,Nadia,21,17
7,Luqman,Dina,Isha,Hazwan,9,21
8,Mujahid,Lisa,Yassier,Zulhakim,10,21
9,Luqman,Dina,Rushdi,Nadia,13,21


File exists! Uploading...
Updated ./matches/2024-10-31_match_results.csv in badgerminton/badgerminton-data!


...

,player_a_1,player_a_2,player_b_1,player_b_2,score_a,score_b
0,Mujahid,Faiz,Ammar,Shazwan,15,21
1,Ammar,Mujahid,Mirza,Shazwan,18,21
2,Isha,Rushdi,Afiqah,Faiz,21,19
3,Luqman,Suhayl,Suraya,Ammar,9,21
4,Mirza,Rushdi,Khairul,Isha,19,21
5,Afiqah,Khairina,Suraya,Mujahid,18,21
6,Khairul,Luqman,Shazwan,Hidayat,21,16
7,Rushdi,Faiz,Isha,Suhayl,16,21
8,Ammar,Afiqah,Khairul,Mirza,21,10
9,Shazwan,Khairina,Luqman,Hidayat,15,21


File exists! Uploading...
Updated ./matches/2025-01-16_match_results.csv in badgerminton/badgerminton-data!


,player_a_1,player_a_2,player_b_1,player_b_2,score_a,score_b
0,Lina,Mirza,Luqman,Haziq,21,19
1,Hasanah,Adib,Mujahid,Shazwan,21,14
2,Hazwan,Nadia,Afiqah,Ammar,15,21
3,Haziq,Mujahid,Luqman,Lina,21,14
4,Hasanah,Mirza,Adib,Shazwan,19,21
5,Afiqah,Nadia,Ammar,Lina,15,21
6,Afiqah,Hazwan,Adib,Hasanah,21,12
7,Lina,Luqman,Lisa,Haziq,21,16
8,Ammar,Nadia,Khairul,Shazwan,21,17
9,Luqman,Mirza,Alif,Mujahid,21,18


File exists! Uploading...
Updated ./matches/2025-01-23_match_results.csv in badgerminton/badgerminton-data!


,player_a_1,player_a_2,player_b_1,player_b_2,score_a,score_b
0,Haziq,Lina,Rushdi,Omar,19,21
1,Afiqah,Fazli,Yassier,Ammar,21,8
2,Mujahid,Ilham,Aidi,Nadia,21,14
3,Rushdi,Omar,Ilham,Ammar,21,17
4,Hasanah,Afiq,Lina,Haziq,8,21
5,Haziq,Mujahid,Lina,Fazli,12,21
6,Nadia,Omar,Rushdi,Ilham,21,15
7,Mujahid,Afiq,Hasanah,Ammar,17,21
8,Aidi,Lina,Afiqah,Fazli,14,21
9,Rushdi,Omar,Haziq,Ilham,22,20
